In [1]:
# For sending GET requests from the API
import requests
# For saving access tokens and for file management when creating and adding to the dataset
import os
# For dealing with json responses we receive from the API
import json
# For displaying the data after
import pandas as pd
# For saving the response data in CSV format
import csv
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
#To add wait time between requests
import time

pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [2]:
import pandas as pd
df = pd.read_csv('Disinformation_Dozen_Activities.csv')

In [3]:
df.shape

(15647, 35)

In [4]:
df.head()

,coordinates,created_at,hashtags,media,urls,favorite_count,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,lang,place,possibly_sensitive,quote_id,retweet_count,retweet_id,retweet_screen_name,source,text,tweet_url,user_created_at,user_id,user_default_profile_image,user_description,user_favourites_count,user_followers_count,user_friends_count,user_listed_count,user_location,user_name,user_screen_name,user_statuses_count,user_time_zone,user_urls,user_verified
0,NaN,Wed May 20 19:04:38 +0000 2020,Covid,NaN,https://www.charityauctionstoday.com/auctions/...,87,1263183873594347520,NaN,NaN,NaN,en,NaN,False,NaN,25,NaN,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",There are so many people struggling with #Covi...,https://twitter.com/RobertKennedyJr/status/126...,Mon Jul 18 16:22:27 +0000 2011,337808606,False,"Robert F. Kennedy, Jr. is founder, chairman + ...",116,469710,646,2293,"Los Angles, California",Robert F. Kennedy Jr,RobertKennedyJr,5302,NaN,https://childrenshealthdefense.org/defender/,True
1,NaN,Wed May 13 16:01:18 +0000 2020,NaN,NaN,http://ow.ly/j9KO50zEgZb,29,1260601020142489601,NaN,NaN,NaN,en,NaN,False,NaN,21,NaN,NaN,"<a href=""https://www.hootsuite.com"" rel=""nofol...",Researchers at several universities and medica...,https://twitter.com/mercola/status/12606010201...,Tue Jan 22 05:37:28 +0000 2008,12524522,False,"Osteopathic physician, best-selling author, an...",1653,361271,28302,5714,"Cape Coral, FL",Dr. Joseph Mercola,mercola,81328,NaN,http://dlvr.it/S5SBZN,False
2,NaN,Sun May 31 15:10:25 +0000 2020,NaN,NaN,NaN,0,1267111194164412417,NaN,NaN,NaN,en,NaN,NaN,NaN,12,1.266743e+18,ClimateEFund,"<a href=""https://mobile.twitter.com"" rel=""nofo...","Yes! ""Right now, real leaders should be doubli...",https://twitter.com/RobertKennedyJr/status/126...,Mon Jul 18 16:22:27 +0000 2011,337808606,False,"Robert F. Kennedy, Jr. is founder, chairman + ...",116,469710,646,2293,"Los Angles, California",Robert F. Kennedy Jr,RobertKennedyJr,5302,NaN,https://childrenshealthdefense.org/defender/,True
3,NaN,Sun May 10 04:06:23 +0000 2020,NaN,NaN,NaN,0,1259333939589775362,NaN,NaN,NaN,en,NaN,NaN,NaN,7756,1.258126e+18,delbigtree,"<a href=""http://twitter.com/download/iphone"" r...",WHAT WOULD YOU DO if your six year old son or ...,https://twitter.com/DrChrisNorthrup/status/125...,Tue Feb 16 17:33:01 +0000 2010,114807372,False,Dr. Christiane Northrup is a leading authority...,75,114775,594,1704,Maine,Christiane Northrup,DrChrisNorthrup,12072,NaN,http://www.drnorthrup.com,False
4,NaN,Sat Feb 08 00:00:54 +0000 2020,NaN,NaN,http://ow.ly/SrBE50yfE5K,61,1225932480819138561,NaN,NaN,NaN,en,NaN,False,NaN,33,NaN,NaN,"<a href=""https://www.hootsuite.com"" rel=""nofol...",Loneliness is at epidemic proportions in the U...,https://twitter.com/mercola/status/12259324808...,Tue Jan 22 05:37:28 +0000 2008,12524522,False,"Osteopathic physician, best-selling author, an...",1653,361271,28302,5714,"Cape Coral, FL",Dr. Joseph Mercola,mercola,81328,NaN,http://dlvr.it/S5SBZN,False


In [5]:
# Get the recipients and remove the nan from the beginning for blanks
recipients = df["in_reply_to_screen_name"].unique()[1:]
recipients.sort()
recipients.size

306

In [6]:
# Get the recipients and remove the nan from the beginning for blanks
half_dozen = df["user_screen_name"].unique()
half_dozen.sort()
half_dozen

array(['DrButtar', 'DrChrisNorthrup', 'RobertKennedyJr', 'kevdjenkins1',
       'mercola', 'sayerjigmi'], dtype=object)

In [7]:
half_dozen_dict = {}
for dude in half_dozen:
    dude_recipients = df.loc[df['user_screen_name'] == dude]
    dude_recipients = dude_recipients['in_reply_to_screen_name'].unique()[1:]
    dude_recipients.sort()
    half_dozen_dict[dude] = dude_recipients

In [8]:
half_dozen_dict

{'DrButtar': array(['2600hz', '7dancingrain7', 'Achilles12219', 'AldnoahVoid',
        'AzamHussainUK', 'BayouGirl54', 'BelleElle82', 'BrittElise727',
        'CJforeveryoung', 'CarolM39', 'Corbynsmom7', 'DRockxxxx',
        'DarlaElaine0406', 'DollarrrrJim', 'DonCarlitos', 'DrButtar',
        'Gary133q', 'Gerrieag', 'Googoolygaga', 'HerpnutPat', 'JacquiJ13',
        'JimGlen3', 'JimW1967', 'Jmanuel7101', 'JonKeel',
        'JonMoor62699896', 'JoshAtDrButtar', 'JumbleWumbles',
        'KayBeck03904159', 'Kellie23364945', 'LairdDourka',
        'LindaForTrump1', 'MaryamHenein', 'McmaxClaire', 'MeliSa48999225',
        'Msblowana', 'MyxaZZ', 'NSolomonfair', 'Nehanotknown',
        'PassionateNana', 'PerryAlexJ', 'Qtherise', 'QueenRania',
        'RebelXentric', 'RepKenBuck', 'SharylAttkisson', 'SwineFluHeds',
        'SwineFluNewsIN', 'TannersDad', 'TexG11', 'TheFuknPunisher',
        'ThisBastard', 'Tishestrada1988', 'TransparentMed1', 'TryingWake',
        'Tuenchai33', 'Vinod65001857'

In [9]:
[half_dozen_dict[x].size for x in half_dozen]

[112, 105, 8, 26, 38, 22]

In [10]:
os.environ['BEARER_TOKEN'] = 'AAAAAAAAAAAAAAAAAAAAAPivjAEAAAAA9Iq8L4zATkdo8WdJ4QXA0urpMC0%3DCvMegWX84dRezrZ4pdCxVAUEzOVy3ohpesBk1O34mj38B8VT3X'

In [11]:
import requests
import os
import json

# To set your enviornment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'
bearer_token = os.environ.get("BEARER_TOKEN")


def create_url(accounts):
    # Specify the usernames that you want to lookup below
    # You can enter up to 100 comma-separated values.
    usernames = "usernames=" + accounts
    user_fields = "user.fields=id,name,url,username,location,description,verified,created_at,profile_image_url,public_metrics,protected,withheld"
    # User fields are adjustable, options include:
    # created_at, description, entities, id, location, name,
    # pinned_tweet_id, profile_image_url, protected,
    # public_metrics, url, username, verified, and withheld
    url = "https://api.twitter.com/2/users/by?{}&{}".format(usernames, user_fields)
    return url


def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2UserLookupPython"
    return r


def connect_to_endpoint(url):
    response = requests.request("GET", url, auth=bearer_oauth,)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()


# def main():
#     url = create_url()
#     json_response = connect_to_endpoint(url)
#     print(json.dumps(json_response, indent=4, sort_keys=True))


# if __name__ == "__main__":
#     main()

In [12]:
api_limit = 100
user_list = []
error_list = []
for i in range(recipients.size // api_limit + 1):
    users = ','.join(recipients[(i*api_limit):((i+1)*api_limit)])
    url = create_url(users)
    response = connect_to_endpoint(url)
    user_list = user_list + response["data"]
    error_list = error_list + response["errors"]

200
200
200
200


In [13]:
for user in user_list:
    user["followers_count"] = user["public_metrics"]["followers_count"]
    user["following_count"] = user["public_metrics"]["following_count"]
    user["tweet_count"] = user["public_metrics"]["tweet_count"]
    user["listed_count"] = user["public_metrics"]["listed_count"]
    user.pop('public_metrics', None)

In [14]:
user_list

[{'username': '1ThessCh5',
  'name': 'Sarah Grace',
  'verified': False,
  'id': '1241472932456599552',
  'description': 'JESUS IS KING OF KINGS, AND LORD OF LORDS 🥰',
  'protected': False,
  'location': 'Black Mountain, NC ',
  'created_at': '2020-03-21T21:13:16.000Z',
  'profile_image_url': 'https://pbs.twimg.com/profile_images/1553078898866491392/4sJQhPqZ_normal.jpg',
  'followers_count': 24905,
  'following_count': 21376,
  'tweet_count': 38458,
  'listed_count': 26},
 {'username': '2600hz',
  'name': '2600hz 🇧🇷🇧🇷🇧🇷',
  'verified': False,
  'id': '73229721',
  'description': '“Rage, rage against the dying of the light of consciousness.”',
  'protected': False,
  'location': 'Brasil',
  'created_at': '2009-09-10T21:47:26.000Z',
  'profile_image_url': 'https://pbs.twimg.com/profile_images/1248638909468286976/4jr9zYmL_normal.jpg',
  'followers_count': 917,
  'following_count': 1649,
  'tweet_count': 2945,
  'listed_count': 19},
 {'username': '7dancingrain7',
  'name': '.....',
  'veri

In [15]:
len(user_list)

241

In [16]:
len(error_list)

65

In [17]:
users_df = pd.DataFrame(user_list)

In [18]:
users_df[users_df["verified"] == True]

,username,name,verified,id,description,protected,location,created_at,profile_image_url,followers_count,following_count,tweet_count,listed_count,url
4,AlecBaldwin,AlecBaldwin(HABF),True,288452305,Hilaria and Alec Baldwin Foundation. Supportin...,True,"New York, NY",2011-04-26T21:45:30.000Z,https://pbs.twimg.com/profile_images/147415903...,1032141,2101,29982,8006,https://t.co/RyEIVjgbRQ
5,AlexBerenson,Alex Berenson,True,87679233,Author of Tell Your Children and Pandemia. Bac...,False,New York,2009-11-05T12:58:35.000Z,https://pbs.twimg.com/profile_images/128821533...,425653,208,19700,2571,https://t.co/B75By8xZ7F
8,AndrewRamosTV,Andrew Ramos,True,33768456,People tell me their stories. I put it on TV. ...,False,NY | NJ,2009-04-21T02:06:33.000Z,https://pbs.twimg.com/profile_images/124568562...,3731,854,6290,72,https://t.co/aFhm4VA7Pl
10,AnitaMoorjani,Anita Moorjani,True,199937833,"Author, speaker, & member of the human race. 🌈...",False,Planet Earth,2010-10-08T02:13:41.000Z,https://pbs.twimg.com/profile_images/830184627...,28621,156,8332,323,https://t.co/Su75w89RBR
15,BetoORourke,Beto O'Rourke,True,342863309,,False,"El Paso, TX",2011-07-26T18:05:52.000Z,https://pbs.twimg.com/profile_images/117772514...,2591633,1642,13250,7798,https://t.co/Dz2B5OUIxb
20,BrendanCarrFCC,Brendan Carr,True,2651190312,"Commissioner, Federal Communications Commissio...",False,Washington DC,2014-07-16T14:23:50.000Z,https://pbs.twimg.com/profile_images/158071000...,39653,2775,7149,312,https://t.co/eE6tlQ9H91
22,Bruno_J_Navarro,Bruno J. Navarro,True,151603912,"https://t.co/YfszM2PrMe | newsroom veteran, fl...",False,"New York, USA",2010-06-03T20:16:06.000Z,https://pbs.twimg.com/profile_images/152550596...,7349,9999,211650,414,https://t.co/vRCAmjg9Q4
23,CCDHate,Center for Countering Digital Hate,True,1121799138239111168,International organization disrupting the prod...,False,Washington DC,2019-04-26T15:32:11.000Z,https://pbs.twimg.com/profile_images/152685470...,27619,2131,4255,510,https://t.co/aNH0LOEG5W
25,CharlesMBlow,Charles M. Blow,True,20772763,@NYTimes columnist. @MSNBC political analyst. ...,False,Brooklyn,2009-02-13T14:17:54.000Z,https://pbs.twimg.com/profile_images/145321505...,630774,1721,68448,7021,https://t.co/wQhcQuEWUH
26,CherylHines,Cheryl Hines,True,1099332883,"actress. Flight Attendant S2, Curb Your Enthus...",False,Los Angeles,2013-01-17T21:29:46.000Z,https://pbs.twimg.com/profile_images/151231569...,35026,192,3667,166,https://t.co/sPfY5XcjU9


In [19]:
users_df.to_csv("disinfo12_replies.csv")

In [20]:
error_df = pd.DataFrame(error_list)
error_df.to_csv("disinfo12_errors.csv")

In [34]:
ms6 = ['DrButtar',
 'DrChrisNorthrup',
 'RobertKennedyJr',
 'kevdjenkins1',
 'mercola',
 'sayerjigmi'] 

api_limit = 100
user_list = []
error_list = []
for i in range(1):
    users = ','.join(ms6)
    url = create_url(users)
    response = connect_to_endpoint(url)
    user_list = user_list + response["data"]

200


In [35]:
ms6

['DrButtar',
 'DrChrisNorthrup',
 'RobertKennedyJr',
 'kevdjenkins1',
 'mercola',
 'sayerjigmi']

In [36]:
user_list

[{'public_metrics': {'followers_count': 87815,
   'following_count': 29,
   'tweet_count': 3161,
   'listed_count': 285},
  'name': 'Dr Rashid A Buttar, FAAPM, FACAM, FAAIM',
  'id': '64423855',
  'protected': False,
  'created_at': '2009-08-10T15:03:02.000Z',
  'profile_image_url': 'https://pbs.twimg.com/profile_images/2137996020/buttarhead_normal.jpg',
  'description': "Our Mission:Making the Change the World is waiting for. We've effectively treated patients from 93 countries who had been given up on. Do Right and fear NO man.",
  'location': 'Charlotte, North Carolina, USA',
  'username': 'DrButtar',
  'url': 'https://t.co/YhVYmHNIlE',
  'verified': False},
 {'public_metrics': {'followers_count': 114342,
   'following_count': 596,
   'tweet_count': 12097,
   'listed_count': 1693},
  'name': 'Christiane Northrup',
  'id': '114807372',
  'protected': False,
  'created_at': '2010-02-16T17:33:01.000Z',
  'profile_image_url': 'https://pbs.twimg.com/profile_images/1545770504/NorthrupHead

### Save all the profile pics locally

In [30]:
import urllib.request
import os

os.getcwd()

for user in user_list:
    # Commented out since we already downloaded them
    #urllib.request.urlretrieve(user['profile_image_url'], "img/profile_pics/" + user['username']+".jpg")

'/Users/iankelk/Google Drive/Viz/python'